### Handle ETH Canopy Height dataset with Zampy
Demo notebook for users and developers.

Import packages and configure paths.

In [1]:
import numpy as np
from zampy.datasets.catalog import EthCanopyHeight
from zampy.datasets.dataset_protocol import TimeBounds, SpatialBounds
from pathlib import Path

work_dir = Path("/path_to_work_directory")
download_dir = work_dir / "download"
ingest_dir = work_dir / "ingest"
times = TimeBounds(np.datetime64("2020-01-01"), np.datetime64("2020-12-31"))
bbox_demo = SpatialBounds(54, 6, 51, 3)

Download dataset.

In [2]:
canopy_height_dataset = EthCanopyHeight()
canopy_height_dataset.download(
    download_dir=download_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["height_of_vegetation"],
)

File 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.tif' already exists, skipping...


True

Data ingestion to the unified format in `zampy`.

In [3]:
canopy_height_dataset.ingest(download_dir, ingest_dir)

File 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map.nc' already exists, skipping...
File 'ETH_GlobalCanopyHeight_10m_2020_N48E003_Map.nc' already exists, skipping...
File 'ETH_GlobalCanopyHeight_10m_2020_N48E003_Map_SD.nc' already exists, skipping...
File 'ETH_GlobalCanopyHeight_10m_2020_N51E003_Map_SD.nc' already exists, skipping...


True

In [4]:
ds = canopy_height_dataset.load(
    ingest_dir=ingest_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["height_of_vegetation"],
    resolution=0.05,
)

In [5]:
ds

<xarray.Dataset>
Dimensions:               (time: 1, latitude: 61, longitude: 61)
Coordinates:
  * time                  (time) datetime64[ns] 2020-07-01
  * latitude              (latitude) float64 51.0 51.05 51.1 ... 53.9 53.95 54.0
  * longitude             (longitude) float64 3.0 3.05 3.1 3.15 ... 5.9 5.95 6.0
Data variables:
    height_of_vegetation  (time, latitude, longitude) float32 dask.array<chunksize=(1, 61, 61), meta=np.ndarray>

In [6]:
from zampy.datasets import converter

ds_convert = converter.convert(ds, canopy_height_dataset, "ALMA")

height_of_vegetation renamed to Hveg.


/tmp/ipykernel_21227/3985842214.py:3: UserWarning: All variables already follow the ALMA convention or not included in the ALMA convention.
No conversion operation was performed on 'eth-canopy-height'.
  ds_convert = converter.convert(ds, canopy_height_dataset, "ALMA")


For testing purpose only. <br>
Since the canopy height dataset doesn't need to have a unit conversion performed, we just fake a dataset to trigger the conversion step.

In [7]:
# concerning the memory limit, we take a subset for testing
ds_test = ds.sel(latitude=slice(51, 52), longitude=slice(3.0,4.0))

In [8]:
ds_test

<xarray.Dataset>
Dimensions:               (time: 1, latitude: 21, longitude: 21)
Coordinates:
  * time                  (time) datetime64[ns] 2020-07-01
  * latitude              (latitude) float64 51.0 51.05 51.1 ... 51.9 51.95 52.0
  * longitude             (longitude) float64 3.0 3.05 3.1 3.15 ... 3.9 3.95 4.0
Data variables:
    height_of_vegetation  (time, latitude, longitude) float32 dask.array<chunksize=(1, 21, 21), meta=np.ndarray>

In [9]:
ds_test["Latent_heat_flux"] = ds_test["height_of_vegetation"] * 0.5
ds_test["Latent_heat_flux"].attrs["units"] = "watt/decimeter**2"
ds_test

<xarray.Dataset>
Dimensions:               (time: 1, latitude: 21, longitude: 21)
Coordinates:
  * time                  (time) datetime64[ns] 2020-07-01
  * latitude              (latitude) float64 51.0 51.05 51.1 ... 51.9 51.95 52.0
  * longitude             (longitude) float64 3.0 3.05 3.1 3.15 ... 3.9 3.95 4.0
Data variables:
    height_of_vegetation  (time, latitude, longitude) float32 dask.array<chunksize=(1, 21, 21), meta=np.ndarray>
    Latent_heat_flux      (time, latitude, longitude) float32 dask.array<chunksize=(1, 21, 21), meta=np.ndarray>

In [10]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.33 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39905,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.33 GiB
Comm: tcp://127.0.0.1:32795,Total threads: 2
Dashboard: http://127.0.0.1:36437/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:35425,


In [11]:
ds_convert = converter.convert(ds_test, canopy_height_dataset, "ALMA")

height_of_vegetation renamed to Hveg.
Latent_heat_flux renamed to Qle.
Conversion of dataset 'eth-canopy-height' following ALMA convention is complete!


In [12]:
ds_convert.compute()

<xarray.Dataset>
Dimensions:    (time: 1, latitude: 21, longitude: 21)
Coordinates:
  * time       (time) datetime64[ns] 2020-07-01
  * latitude   (latitude) float64 51.0 51.05 51.1 51.15 ... 51.9 51.95 52.0
  * longitude  (longitude) float64 3.0 3.05 3.1 3.15 3.2 ... 3.85 3.9 3.95 4.0
Data variables:
    Hveg       (time, latitude, longitude) float32 nan 3.569 4.862 ... nan nan
    Qle        (time, latitude, longitude) float32 nan 178.5 243.1 ... nan nan

In [13]:
# check the conversion
assert np.allclose(
    ds_convert["Qle"][0,:20,:20].values / 100,
    ds_test["Latent_heat_flux"][0,:20,:20].values,
    equal_nan=True,
)